In [ ]:
!git clone -q https://github.com/soumik12345/point-cloud-segmentation

In [ ]:
import sys

sys.path.append("point-cloud-segmentation")

from point_seg import ShapeNetCoreLoaderInMemory
from point_seg import models

import tensorflow as tf

from datetime import datetime
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [ ]:
data_loader = ShapeNetCoreLoaderInMemory(
    object_category="Airplane", n_sampled_points=2048
)
data_loader.load_data()
train_dataset, val_dataset = data_loader.get_datasets(batch_size=32)

In [ ]:
x, y = next(iter(train_dataset))
print(x.shape, y.shape)

num_points = x.shape[1]
num_classes = y.shape[-1]

In [ ]:
baseline_segmentation_model = models.get_baseline_segmentation_model(
    num_points, num_classes
)
baseline_segmentation_model.summary()

In [ ]:
initial_lr = 0.001
drop_every = 20
decay_factor = 0.5


# Reference: https://www.pyimagesearch.com/2019/07/22/keras-learning-rate-schedules-and-decay/
def lr_schedule(epoch):
    exp = np.floor((1 + epoch) / drop_every)
    new_lr = initial_lr * (decay_factor ** exp)
    return new_lr


lr_callback = tf.keras.callbacks.LearningRateScheduler(
    lambda epoch: lr_schedule(epoch), verbose=True
)

In [ ]:
def run_experiment(epochs=100):
    optimizer = tf.keras.optimizers.Adam(learning_rate=initial_lr)
    baseline_segmentation_model = models.get_baseline_segmentation_model(
        num_points, num_classes
    )
    baseline_segmentation_model.compile(
        optimizer=optimizer, loss="categorical_crossentropy"
    )

    logs_dir = f'logs_{datetime.utcnow().strftime("%y%m%d-%H%M%S")}'
    tb_callback = tf.keras.callbacks.TensorBoard(log_dir=logs_dir)

    baseline_segmentation_model.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=epochs,
        callbacks=[tb_callback, lr_callback],
    )


run_experiment()